<a href="https://colab.research.google.com/github/matdjohnson-at-umass-dot-edu/cs646-final-project/blob/main/CS646_Final_Project_Cos_Sim_IR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install datasets
! pip install transformers
! pip install pytrec_eval

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
  Preparing metadata (setup.py) ... done
  Created wheel for pytrec_eval: filename=pytrec_eval-0.5-cp310-cp310-linux_x86_64.whl size=308203 sha256=ca27077831d7d

In [ ]:
from datasets import concatenate_datasets, Dataset, disable_caching, disable_progress_bars, load_dataset
from statsmodels.distributions.empirical_distribution import ECDF
from tqdm import tqdm
from google.colab import drive
import os
import torch
import torch.nn.functional as torch_func
import gc
import time
from threading import Lock
from concurrent.futures import ThreadPoolExecutor
from transformers import AutoTokenizer, AutoModel
import logging
import psutil
import numpy as np
from collections import Counter
import pytrec_eval

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
datasets_base_dir = "/content/drive/MyDrive/CS646-FinalProject/datasets"
filtered_examples_dataset_file_name = "ms_marco_corpus_in_qrel_embs_avg_example_ids_filtered"
filtered_examples_file_path_and_name = f"{datasets_base_dir}/{filtered_examples_dataset_file_name}/{filtered_examples_dataset_file_name}.parquet"
final_dataset_test_file_path_and_name = f"{datasets_base_dir}/ms_marco_final_dataset_avg/ms_marco_final_dataset_avg_test.parquet"

qrels_dataset = load_dataset('BeIR/msmarco-qrels')
qrels = {}
for entry in qrels_dataset['train']:
    if qrels.get(str(entry['query-id'])) is None:
        qrels[str(entry['query-id'])] = { str(entry['corpus-id']): int(entry['score']) }
    else:
        qrels[str(entry['query-id'])]['corpus-id'] = int(entry['score'])
for entry in qrels_dataset['test']:
    if qrels.get(str(entry['query-id'])) is None:
        qrels[str(entry['query-id'])] = { str(entry['corpus-id']): int(entry['score']) }
    else:
        qrels[str(entry['query-id'])]['corpus-id'] = int(entry['score'])
for entry in qrels_dataset['validation']:
    if qrels.get(str(entry['query-id'])) is None:
        qrels[str(entry['query-id'])] = { str(entry['corpus-id']): int(entry['score']) }
    else:
        qrels[str(entry['query-id'])]['corpus-id'] = int(entry['score'])
for entry in qrels_dataset['test']:
    if qrels.get(str(entry['query-id'])) is None:
        qrels[str(entry['query-id'])] = { str(entry['corpus-id']): int(entry['score']) }
    else:
        qrels[str(entry['query-id'])]['corpus-id'] = int(entry['score'])

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

preprocessed_dataset = Dataset.from_parquet(filtered_examples_file_path_and_name).with_format("torch", device=device)

doc_ids = torch.concatenate((
    preprocessed_dataset['pos_doc_id'],
    preprocessed_dataset['neg_doc_id']
))
doc_embs = torch.concatenate((
    preprocessed_dataset['pos_doc_emb'],
    preprocessed_dataset['neg_doc_emb']
)).unsqueeze(-1)

del preprocessed_dataset
gc.collect()
torch.cuda.empty_cache()
gc.collect()

preprocessed_dataset_test = Dataset.from_parquet(final_dataset_test_file_path_and_name).with_format("torch", device=device)

query_ids = preprocessed_dataset_test['query_id'].tolist()
query_embs = preprocessed_dataset_test['query_emb'].unsqueeze(-1)

rel_evaluator = pytrec_eval.RelevanceEvaluator(qrels, {'recall.10', 'recall.100', 'recall.1000'})

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/14.0k [00:00<?, ?B/s]

train.tsv:   0%|          | 0.00/9.52M [00:00<?, ?B/s]

dev.tsv:   0%|          | 0.00/136k [00:00<?, ?B/s]

test.tsv:   0%|          | 0.00/169k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/532751 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/7437 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/9260 [00:00<?, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
batch_size = 7
batch_count = (query_embs.shape[0] // batch_size) + 1
log_freq = 15

results = {}

for i in range(0, batch_count):
    lower_bound = i * batch_size
    upper_bound = min( (i+1) * batch_size, query_embs.shape[0] )
    query_embs_segment = query_embs[lower_bound:upper_bound,:,:]
    timestamp = time.strftime("%Y-%m-%dT%H:%M:%S", time.localtime())
    mem_info = torch.cuda.mem_get_info(device)
    if i % log_freq == 0 or i ==0:
        timestamp = time.strftime("%Y-%m-%dT%H:%M:%S", time.localtime())
        mem_info = torch.cuda.mem_get_info(device)
        print(f"{timestamp}: batch {i+1} of {batch_count} - pre-cosine computation - gpu_usage: {(mem_info[1] - mem_info[0]) / 1024 / 1024 / 1024}")
        # print(f"{timestamp}: doc_ids.shape {doc_ids.shape}")
        # print(f"{timestamp}: doc_embs.shape {doc_embs.shape}")
        # print(f"{timestamp}: query_embs.shape {query_embs.shape}")
        # print(f"{timestamp}: query_embs_segment.shape {query_embs_segment.shape}")
    embs_cos_sim = torch_func.cosine_similarity(
        doc_embs,
        query_embs_segment.T
    )
    if i % log_freq == 0 or i ==0:
        timestamp = time.strftime("%Y-%m-%dT%H:%M:%S", time.localtime())
        mem_info = torch.cuda.mem_get_info(device)
        print(f"{timestamp}: batch {i+1} of {batch_count} - post-cosine computation - gpu_usage: {(mem_info[1] - mem_info[0]) / 1024 / 1024 / 1024}")
    k=1000
    top_k = torch.topk(
        embs_cos_sim,
        k,
        dim=0
    )
    # if i % log_freq == 0 or i ==0:
    #     print(f"{timestamp}: embs_cos_sim.shape {embs_cos_sim.shape}")
    #     print(f"{timestamp}: top_k[0].shape, top_k[1].shape {top_k[0].shape, top_k[1].shape}")
    top_k_ind_for_take = top_k[1].T.flatten()
    top_k_doc_ids = torch.reshape(doc_ids.take(top_k_ind_for_take), (-1, k))
    for j in range(lower_bound, upper_bound):
        batch_index = j - lower_bound
        results[str(query_ids[j])] = dict(zip(
            map(lambda element: str(element), top_k_doc_ids[batch_index,:].detach().cpu().tolist()),
            top_k[0].T[batch_index,:].detach().cpu().tolist()
        ))
    del query_embs_segment, embs_cos_sim, top_k, top_k_ind_for_take, top_k_doc_ids
    gc.collect()
    torch.cuda.empty_cache()
    gc.collect()

rel_evalution = rel_evaluator.evaluate(results)
print(f"rel_evalution: {rel_evalution}"[:1500])


2024-12-05T09:43:35: batch 1 of 1515 - pre-cosine computation - gpu_usage: 2.0966796875
2024-12-05T09:43:35: doc_ids.shape torch.Size([423992])
2024-12-05T09:43:35: doc_embs.shape torch.Size([423992, 1024, 1])
2024-12-05T09:43:35: query_embs.shape torch.Size([10600, 1024, 1])
2024-12-05T09:43:35: query_embs_segment.shape torch.Size([7, 1024, 1])


<ipython-input-4-b429b86794a6>:24: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3683.)
  query_embs_segment.T


Streaming output truncated to the last 5000 lines.
2024-12-05T10:00:44: doc_ids.shape torch.Size([423992])
2024-12-05T10:00:44: doc_embs.shape torch.Size([423992, 1024, 1])
2024-12-05T10:00:44: query_embs.shape torch.Size([10600, 1024, 1])
2024-12-05T10:00:44: query_embs_segment.shape torch.Size([7, 1024, 1])
2024-12-05T10:00:45: batch 891 of 1515 - post-cosine computation - gpu_usage: 36.1591796875
2024-12-05T10:00:45: embs_cos_sim.shape torch.Size([423992, 7])
2024-12-05T10:00:45: top_k[0].shape, top_k[1].shape (torch.Size([1000, 7]), torch.Size([1000, 7]))
2024-12-05T10:00:46: batch 892 of 1515 - pre-cosine computation - gpu_usage: 2.1689453125
2024-12-05T10:00:46: doc_ids.shape torch.Size([423992])
2024-12-05T10:00:46: doc_embs.shape torch.Size([423992, 1024, 1])
2024-12-05T10:00:46: query_embs.shape torch.Size([10600, 1024, 1])
2024-12-05T10:00:46: query_embs_segment.shape torch.Size([7, 1024, 1])
2024-12-05T10:00:46: batch 892 of 1515 - post-cosine computation - gpu_usage: 36.159

In [ ]:
recall_10_sum = 0
recall_100_sum = 0
recall_1000_sum = 0

for query_id in rel_evalution.keys():
    recall_10_sum = recall_10_sum + rel_evalution[query_id]['recall_10']
    recall_100_sum = recall_100_sum + rel_evalution[query_id]['recall_100']
    recall_1000_sum = recall_1000_sum + rel_evalution[query_id]['recall_1000']

average_recall_10 = recall_10_sum / len(rel_evalution.keys())
average_recall_100 = recall_100_sum / len(rel_evalution.keys())
average_recall_1000 = recall_1000_sum / len(rel_evalution.keys())

print(f"average_recall_10: {average_recall_10}, average_recall_100: {average_recall_100}, average_recall_1000: {average_recall_1000}")

average_recall_10: 0.4824175824175824, average_recall_100: 0.7279025322503584, average_recall_1000: 0.8584806497849976
